## How to Use This Notebook

This Jupyter notebook uses the `ipywidgets` module to create a graphical user interface. Here's how to use it:

1. **Run the interface**: The cell below imports the `pygui` module and calls the `create_interface` function. This function creates the graphical user interface. To run this cell, click on it to select it, then click the "Run" button or use the `Shift + Enter` shortcut or the Play button in the top menu.

2. **First steps**: To import data , you need to first replace `user` with your Solforge Fusion `username` in the box below. Then click the `Load` button on the bottom of the notebook. 

In [ ]:
import pygui as pgui
pgui.setup_interface()
pgui.db_list.value = 'tmind'
pgui.tab.selected_index = 1


#TODO: Fix the operator logic 
#TODO: Let the CSSManager handle the css attributes 